In [36]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGVector
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from PyPDF2 import PdfReader
from openai import OpenAI
import gradio as gr
import time


In [7]:
loader = PyPDFLoader("IEO-Q3-2024-EN.pdf")

In [ ]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [13]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    azure_endpoint=,
    api_key=,
    openai_api_version=
)

In [17]:
username = 
password = 
host = 'pgvector-vllm.postgres.database.azure.com'
port = '5432'
dbname = 'jakarta'

CONNECTION_STRING = f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}'
COLLECTION_NAME = 'jakarta'

In [18]:
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING
)

/home/azureuser/vllmclient/lib/python3.12/site-packages/langchain_community/vectorstores/pgvector.py:487: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = cls(


In [25]:
client = OpenAI(
    api_key="EMPTY",
    base_url='http://104.43.91.226:8000/v1'
)

In [38]:
!curl = http://104.43.91.226:8000/v1/models

curl: (3) URL rejected: Bad hostname
{"object":"list","data":[{"id":"meta-llama/Llama-3.1-8B-Instruct","object":"model","created":1731158457,"owned_by":"vllm","root":"meta-llama/Llama-3.1-8B-Instruct","parent":null,"max_model_len":131072,"permission":[{"id":"modelperm-19fa43bc61324a1faf4a0d91cb168774","object":"model_permission","created":1731158457,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [34]:
retriever = db.as_retriever()

In [ ]:
def user(user_prompt, history):
    return "", history + [[user_prompt, None]]

def generate_response(rag_selection, history):
    user_message = history[-1][0]

    if rag_selection == "Direct Output with LLM":
      response = [{"role": "user", "content": user_message}]
    else:
      retrieved_documents = retriever.invoke(user_message)
      context = ' '.join([retrieved_documents[i].page_content for i in range(len(retrieved_documents))])
      prompt = """You are a financial assistant that specialise on Indonesia economy. Your taks is to answer the user's question to the best of your ability.\
                  Use the following pieces of retrieved context to answer the question only if the retrieved context is highly relevant. \
                  If the retrieved context is not relevant, answer the question to your best ability. \

                  {context}"""

      response = [{"role": "system", "content": prompt},
                  {"role": "user", "content": user_message}]

    completion = client.chat.completions.create(
        model='meta-llama/Llama-3.1-8B-Instruct',  # Model name to use
        messages=response,  # Chat history
        stream=True,
        stream_options={"include_usage": True},
        max_tokens=128,  # Maximum number of tokens to generate
        temperature=0.8,  # Temperature for text generation
    )

    bot_response = ""
    tokens_num = 0
    st = time.time()

    for chunk in completion:
      if hasattr(chunk.choices[0], 'finish_reason') and chunk.choices[0].finish_reason is not None:
        next_chunk = next(completion, None)
        if next_chunk:
          tokens_num = next_chunk.usage.completion_tokens
          et = time.time() - st
          yield history, f'{round(tokens_num/et,2)}'
        else:
          return None # Handle the case where there's no next chunk
      else:
        et = time.time() - st
        chunk_message = chunk.choices[0].delta.content or ""
        bot_response += chunk_message
        history[-1][1] = bot_response
        yield history, f'{round(tokens_num/et,2)}'

with gr.Blocks(theme=gr.Theme.from_hub('HaleyCH/HaleyCH_Theme'), css=".column-form .wrap {flex-direction: column;}") as demo:
    with gr.Row():
      gr.Markdown("""
            <h1><center>Intel Chatbot with vLLM Model Serving
            <center><img src="https://upload.wikimedia.org/wikipedia/commons/6/64/Intel-logo-2022.png" width=200px>
            <h2>Inferenced on Azure instance powered by Intel 4th Gen Xeon with AMX Acceleration</h2></n>Using Meta Llama 3.1 8B</center>
            """)
    with gr.Row():
        with gr.Column(visible=True, min_width=250, scale=0) as sidebar:
          with gr.Row():
            gr.Markdown("""
              This is a Chatbot that is supported by additional context from PDF<br>""")
          with gr.Row():
            dropdown = gr.Dropdown(["Direct Output with LLM", "PDF-Augmented Output"], label="Retrieval-Augmented Generation")
          with gr.Row():
            gr.Markdown("""    
              - By default, the chatbot will use the base model.
              - If PDF-Augmented Output is selected, the system will answer based on the PDF<br>              
              """)
          with gr.Row():
            gr.Markdown("""
            **Brochure:**<br>
            **Indonesia Economic Outlook Q3 '24**<br>
            <img src="https://github.com/user-attachments/assets/85ed6ac7-1018-46b4-915a-e856c46a04c9" width=150px>
            """)

        with gr.Column() as main:
            with gr.Row():
                chatbot = gr.Chatbot(label="Conversation")
            with gr.Row():
                prompt_input = gr.Textbox(label="Enter your message here", placeholder="Type your message...",scale=7)
                token_output = gr.Textbox(label="Tokens/sec", placeholder="0.00",scale=3)

                prompt_input.submit(user, [prompt_input, chatbot], [prompt_input, chatbot], queue=False).then(
                generate_response, inputs=[dropdown,chatbot], outputs=[chatbot,token_output], queue=True)

        with gr.Column(visible=True, min_width=250, scale=0) as examplelist:
            gr.Examples(
            examples=[
            ["Tell me about Indonesia Economic Outlook Q3 2024"],
            ["How is Indonesia Economic"],
        ],
        inputs=prompt_input,
    )



    with gr.Row():
            gr.Markdown("""
            <center><br><h3>An Intel Collaboration with Beny Ibrani // Malcolm Chan</h3>
            """)

demo.queue(default_concurrency_limit=100).launch(debug=True, share=True)

/home/azureuser/vllmclient/lib/python3.12/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://35ce0ff4016b578599.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
